In [3]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import re
import pandas as pd
import time
import pickle

In [2]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}


In [3]:
def get_links(year, month:str, page):
    url ='http://www.arthub.co.kr/sub01/board033_list.htm?'
    seed = 'http://www.arthub.co.kr/sub01/'
    
    params = {
        'k_date':year,
        'k_month':month,
        'PHPSESSID':'e124fc854822909d9ec3618c1e6089f2',
        'page':page
        }
    
    resp = requests.get(url,params=params,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    aList = soup.select('a.sub01_list_title')
    
    return [(str(year)+month,a.get_text(),seed+a['href']) for a in aList]

In [37]:
get_links(2018,'09',1)

[('201809',
  '이재건展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=25924'),
 ('201809',
  'An unfamiliar gaze - 정의철展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26876'),
 ('201809',
  '조성희展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=25935'),
 ('201809',
  '서근범展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26893')]

html 태그가 매우 드럽게 달려있다

부적절한 tree 구조인데다가 identification 규칙이 일정치 않음, 일일이 찾기힘듦

In [277]:
artists = [t3.text for t3 in t3.find('td',{'width':'339'}).find_all('a')]
date = [el for el in t3.find('span',{'class':'red_point'}).parent]
time = [el.text for el in t3.find_all('span',{'class':'red_point'})[1:]]

전시정보를 가져오는 함수 get_info를 짰는데 next.next.next... 이런식으로 드럽다, 어쩔수없었다

In [4]:
def get_info(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    t3 = soup.select('table')[3]

    info = []
    for el in [el.next.next.next.next.next for el in t3.find_all('td',{'width':'84','height':'18','align':'left'})]: #전시정보 뽑아오기, tag가 드러움
        if el.string==None: #class 'bs4.element.Tag'
            info.append(el.text)
        else:                          #class 'bs4.element.NavigableString'
            info.append(el.string) 

    artists = list(map(lambda s:s.strip(), info[0].split(',')))
    articles = [td.next['href'] for td in t3.find_all('td',{'width':'510'})[1:]]

    del info[0]
    info.insert(0,artists)
    info.append(articles)
    
    return info

In [231]:
get_info('http://www.arthub.co.kr/sub01/board03_view.htm?No=26567')

[['강지만',
  '김경섭',
  '김용석',
  '김아영',
  '문기전',
  '박정원',
  '박정용',
  '박흥선',
  '성태진',
  '송지연',
  '윤기원',
  '이수정',
  '이재열',
  '진선미'],
 '2018-08-01 ▶ 2018-08-04',
 '없음',
 'Open 14:00 ~ Close 20:00',
 '아트팩토리 후(ART FACTORY hoo)\xa0 다른전시 보기',
 '강원도 원주시 문막읍 비야동길 10-12 (후용리 1318) 나동창고 ',
 '010-4014-7488',
 'www.facebook.com/artfactoryhoo',
 ['http://www.newsis.com/view/?id=NISX20180710_0000359747&cid=10701']]

텍스트를 가져오는 함수 get_text

In [5]:
def get_text(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    ps = [p.text for p in soup.find_all('p',{'align':'justify'})]
    text = ''.join(ps)
    
    return text.strip()

In [238]:
get_text('http://www.arthub.co.kr/sub01/board03_view.htm?No=19282')

'The birth of Life\n        물은 모든 생명의 원천이다. 모든 사물은 물에서 시작되어 그 끝 역시 물로 마무리 되고 그 과정은 지속적인 순환과정을 겪는다. \n        그리고 물의 순환과정을 통해 우리는 생명의 강인함과 진리를 깨닫게 된다. 또한 물에서 비롯된 생명은 하나가 모여 전체가 되듯이 \n        한 화면 속에서 개개의 소재들이 모여 하나의 화면을 구성하며 전일적인 개체로 성장한다.\n\n\n        The song of Life\n        모든 생명은 아름답고 작가의 작업은 그 아름다움을 노래하기 위한 수단이다. 그리고 그 아름다움을 만들어내는 모든 생명의 기원은 \n        물이다. 그래서 그의 작품 속에는 물을 상징하는 소재들이 그려져 있다. 또한 물위에서 기원하듯 그 주위에는 생명력을 품은 아름다운 \n        꽃들이 둘러싸며 저마다의 생명력을 뿜어내고 있다.The \n        energy of Life\n        생명은 항상 강인함을 지닌다. 어떠한 시련에도 생명의 순환은 결코 멈추지 않는다. 그리고 이것은 생명의 원천인 물이 가지는 에너지이기도 \n        하다. 작가는 이러한 끊임없는 순환의 과정 속의 생명이 가지는 강인함과 아름다움을 노래하기 위해 캔버스 위를 유화와 아크릴을 \n        통해 다채로운 색채로 표현한다. 잔잔한 한 폭의 정물화 인듯하지만 화려하고 과감한 색채의 대비는 우리에게 생명의 에너지를 속삭인다. \n         ⓒ'

샘플 데이터를 만들어보자, 55개의 전시만 가져왔다

In [1]:
start = time.time()

arts = []
for y in range(2008,2014):
    for m in tqdm(range(1,13)):
        if m<10:
            m = '0'+str(m)
        else:
            m = str(m)
        m_links = []
        for p in range(1,6): 
            try:
                for _ in range(2): #TimeoutError 방지위해 두번까지 시도한다
                    m_links.extend(get_links(y,m,p))
                    break
            except Exception as e:
                print(e)
                time.sleep(1)
                continue
                
        m_links = list(set(m_links)) #중복 링크 제거
        for link in m_links:
            try:
                info = get_info(link[2])
                text = get_text(link[2])

                temp = [*link,*info,text]
                arts.append(temp)
                
            except Exception as e:
                print(e)
                continue
                
end = time.time()
data = pd.DataFrame(arts,columns=['연월','전시제목','링크','전시작가','전시일정','초대일시','관람시간','전시장소','주소','연락처','홈페이지','아티클','텍스트'])
data.sample(5)

print(end-start)

In [2]:
len(data.전시제목.unique())

In [51]:
data[data.duplicated(subset='전시제목')]

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18309 entries, 0 to 18308
Data columns (total 13 columns):
연월      18309 non-null object
전시제목    18309 non-null object
링크      18309 non-null object
전시작가    18309 non-null object
전시일정    18309 non-null object
초대일시    18309 non-null object
관람시간    18308 non-null object
전시장소    18308 non-null object
주소      18308 non-null object
연락처     18308 non-null object
홈페이지    18308 non-null object
아티클     18292 non-null object
텍스트     18292 non-null object
dtypes: object(13)
memory usage: 1.8+ MB


In [8]:
data.to_excel('test2.xlsx',sheet_name='sheet1')

In [10]:
d = pd.read_excel('test.xlsx',sheet_name='sheet1')

In [11]:
d.sample()

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
13873,200908,장영환展 :: Sculpture,http://www.arthub.co.kr/sub01/board03_view.htm...,['장영환(Chang Younghwan)'],2009-08-19 ▶ 2009-09-08,NaN,Open 10:00 ~ Close 18:00,갤러리현대(GALLERY HYUNDAI) 강남 윈도우갤러리 다른전시 보기,서울시 강남구 신사동 640-6 아트타워,02-519-0800,www.galleryhyundai.com,[''],● \n 장영환展 \n이번이 첫 개인전인 \n 장영환 ...


In [12]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26344 entries, 0 to 26343
Data columns (total 13 columns):
연월      26344 non-null int64
전시제목    26344 non-null object
링크      26344 non-null object
전시작가    26344 non-null object
전시일정    26344 non-null object
초대일시    18971 non-null object
관람시간    26342 non-null object
전시장소    26342 non-null object
주소      26326 non-null object
연락처     26342 non-null object
홈페이지    21614 non-null object
아티클     26326 non-null object
텍스트     25621 non-null object
dtypes: int64(1), object(12)
memory usage: 2.8+ MB


In [13]:
d.dropna(subset=['텍스트'],inplace=True) #텍스트 없는 로우 드랍

In [14]:
sum(d.duplicated(subset=['텍스트'])) # 중복된 텍스트 수 

5101

In [15]:
toDropIndex = d[d.duplicated(subset=['텍스트'])].index

In [16]:
d.drop(index = toDropIndex, inplace=True)

In [17]:
toDropIndex = d[d.duplicated(subset=['전시제목','전시일정'])].index #그래도 같은 전시는 drop

In [18]:
d.drop(index = toDropIndex, inplace=True)

In [19]:
d.info() # 20514 개 데이터

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20514 entries, 0 to 26343
Data columns (total 13 columns):
연월      20514 non-null int64
전시제목    20514 non-null object
링크      20514 non-null object
전시작가    20514 non-null object
전시일정    20514 non-null object
초대일시    16757 non-null object
관람시간    20514 non-null object
전시장소    20514 non-null object
주소      20514 non-null object
연락처     20514 non-null object
홈페이지    17162 non-null object
아티클     20514 non-null object
텍스트     20514 non-null object
dtypes: int64(1), object(12)
memory usage: 2.2+ MB


작가별로 텍스트를 모아보자

In [23]:
artists = {}
for i in d.index:
    try:
        persons = re.sub(r'[\[|\]|\"|\']','',d.iloc[i,:].전시작가).split(',')
        persons = [person.strip() for person in persons]
        for person in persons:
            if person in artists:
                artists[person] += [d.iloc[i,:].텍스트]
            else:
                artists[person] = [d.iloc[i,:].텍스트]
    except Exception as e:
        continue

In [51]:
len(artists)

26404

In [57]:
# for k,v in artists.items():
#     if '(' in k:
#         print(k, len(v))

In [24]:
# with open('artists.dic','wb') as file:
#     pickle.dump(artists,file)

In [25]:
with open('artists.dic','rb') as file:
    artists = pickle.load(file)

In [27]:
artists['강미숙']

['바다의 태양\n        피고 지는 모란꽃\n        빛나는 크리스탈\n        기억의 단편들\n        관계의 연결고리\n        파랑새의 날개짓 그림 속 이야기들이 \n        연결되어 또 하나의 그림이 됩니다. ⓒ',
 '● 여기, 서울展\n        서울,\n        어릴적부터 곳곳에 추억이 담겨져 있는 오래된 앨범같다\n        이미 사라져 버리거나, 너무 많이 변해버린곳도 있지만 그곳을 지날때면 \n        늘 기억속에 머물러 추억을 여행한다\n        화려함에 가려진 고독한 서울의 사람들은 무수한 불빛에 길을 묻고,\n        한강에 눈물을 흘려보내고, 밤하늘 별빛에 다시 꿈을 띄운다\n        우리가 살고 있는 이곳, 서울\n        너무 가까워서, 반복되는 일상이 되어 모르고 지나치던 \n        서울의 공기, 서울의 색깔, 서울의 향기를 느껴보자\n        알고보면 그 어떤 곳보다도 아름답고 로맨틱하고 따뜻한 서울을 여행한다.\n\n오늘, 지금, 여기 당신의 서울은 \n        어떤가요? ⓒ',
 '● 꽃다방에서展\n        꽃다방 작은 창으로 들어오는 겨울의 수줍은 햇살 아래에서']